In [1]:
import fxcmpy
import time
import datetime as dt
import talib as ta
import numpy as np
from array import *

### STRATEGY DESCRIPTION ####
# This strategy buys when the Fast SMA crosses over the Slow SMA and sell when the Fast SMA crosses
# under the Slow SMA. Opposing signals will close out opposing positions if close_on_opposing_signal == True.
# If close_on_opposing_signal == False, the strategy can open hedged positions (long & short at the same time)
# Parameters allow traders to change token, symbol, timeframe, Fast SMA & Slow SMA Periods, 
# Opposing signal closing logic, trade size/stop/limit.
# This is a close-of-bar strategy, meaning it only signals trades at the close of a bar.
# For more strategy examples, please visit github.com/fxcm/RestAPI
#############################

###### USER PARAMETERS ######
token = '817d90e62163c99e1dcf4f03b7cfadd52222ced6'
symbol = 'EUR/USD'
timeframe = "m5"	        # (m1,m5,m15,m30,H1,H2,H3,H4,H6,H8,D1,W1,M1)

close_on_opposing_signal = True
amount = 2
stop = -10
trailstop_value = 10
limit = 50

TrendUp = array('i', [])
TrendDown = array('i', [])
#############################

# Global Variables
pricedata = None
numberofcandles = 300

# Connect to FXCM API
con = fxcmpy.fxcmpy(access_token="", log_level="error")
# con = fxcmpy.fxcmpy(config_file='fxcm.cfg')
	
# This function runs once at the beginning of the strategy to run initial one-time processes/computations
def Prepare():
	global pricedata
	
	print("Requesting Initial Price Data...")
	pricedata = con.get_candles(symbol, period=timeframe, number=numberofcandles)
	print(pricedata)
	print("Initial Price Data Received...")

# Get latest close bar prices and run Update() function every close of bar/candle
def StrategyHeartBeat():
	while True:
		currenttime = dt.datetime.now()
		if timeframe == "m1" and currenttime.second == 0 and GetLatestPriceData():
			Update()
		elif timeframe == "m5" and currenttime.second == 0 and currenttime.minute % 5 == 0 and GetLatestPriceData(): 
			Update()
			time.sleep(240)
		elif timeframe == "m15" and currenttime.second == 0 and currenttime.minute % 15 == 0 and GetLatestPriceData(): 
			Update()
			time.sleep(840)
		elif timeframe == "m30" and currenttime.second == 0 and currenttime.minute % 30 == 0 and GetLatestPriceData():
			Update()
			time.sleep(1740)
		elif currenttime.second == 0 and currenttime.minute == 0 and GetLatestPriceData():
			Update()
			time.sleep(3540)
		time.sleep(1)

# Returns True when pricedata is properly updated			
def GetLatestPriceData():
	global pricedata
	
	# Normal operation will update pricedata on first attempt
	new_pricedata = con.get_candles(symbol, period=timeframe, number=numberofcandles)
	if new_pricedata.index.values[len(new_pricedata.index.values)-1] != pricedata.index.values[len(pricedata.index.values)-1]:
		pricedata= new_pricedata
		return True
		
	counter = 0
	# If data is not available on first attempt, try up to 3 times to update pricedata
	while new_pricedata.index.values[len(new_pricedata.index.values)-1] == pricedata.index.values[len(pricedata.index.values)-1] and counter < 3:
		print("No updated prices found, trying again in 10 seconds...")
		counter+=1
		time.sleep(10)
		new_pricedata = con.get_candles(symbol, period=timeframe, number=numberofcandles)
	if new_pricedata.index.values[len(new_pricedata.index.values)-1] != pricedata.index.values[len(pricedata.index.values)-1]:
		pricedata = new_pricedata
		return True
	else:
		return False
		
# This function is run every time a candle closes
def Update():
	print(str(dt.datetime.now()) + "	 " + timeframe + " Bar Closed - Running Update Function...")
    
    #ALGORITHM
    
	Factor = 3
	Pd= 7
	highs = np.array(pricedata['bidhigh'])
	lows = np.array(pricedata['bidlow'])
	closes = np.array(pricedata['bidclose'])
	hl2 = (pricedata['bidhigh'][len(pricedata)-1] + pricedata['bidlow'][len(pricedata)-1])/2
    
	atr = ta.real = ATR(highs, lows, closes, timeperiod=Pd)
    
	Up= hl2 -(Factor*atr)
	Dn= hl2 +(Factor*atr)
    
	if pricedata['bidclose'][len(pricedata)-1] > TrendUp[len(TrendUp)-1]:
		TrendUp_value = max(Up,TrendUp[1])
	else:
		TrendUp_value = Up
        
	if pricedata['bidclose'][len(pricedata)-1] < TrendDown[len(TrendDown)-1]:
		TrendDown_value = min(Dn,TrendDown[1])
	else:
		TrendDown_value = Dn

	TrendUp.append(TrendUp_value)
	TrendDown.append(TrendDown_value)
        
        
	if pricedata['bidclose'][len(pricedata)-1] > TrendDown[len(TrendDown)-1]
		Trend = 1
	elif close< TrendUp[1]
		Trend = -1

        
        
	if Trend==1: 
        Tsl = TrendUp
    else:
        Tsl = TrendDown

	linecolor = Trend == 1 ? green : red

    
    
    
    #TRADING LOGIC
    #Buy If colFinal is aqua and colFinal2 is lime
	if colFinal == "aqua" and colFinal2 == "lime":
		print("	  BUY SIGNAL!")
		if close_on_opposing_signal and countOpenTrades("S") > 0:
			print("	  Closing Sell Trade(s)...")
			exit("S")
		if countOpenTrades("B") == 0:
			print("	  Opening Buy Trade...")
			enter("B")
        
    #Sell If colFinal is orange and colFinal2 is red
	if colFinal == "orange" and colFinal2 == "red":
		print("	  SELL SIGNAL!")
		if close_on_opposing_signal and countOpenTrades("B") > 0:
			print("	  Closing Buy Trade(s)...")
			exit("B")
		if countOpenTrades("S") == 0:
			print("	  Opening Sell Trade...")
			enter("S")
        
    #EXIT STRTEGY
	if colFinal == "gray":
		print("	  EXIT SELL SIGNAL!")
		if close_on_opposing_signal and countOpenTrades("S") > 0:
			print("	  Closing Sell Trade(s)...")
			exit("S")
		print("	  EXIT BUY SIGNAL!")
		if close_on_opposing_signal and countOpenTrades("B") > 0:
			print("	  Closing Buy Trade(s)...")
			exit("B")
            
    
        
	
	# Print Price/Indicators
	print("Close Price: " + str(pricedata['bidclose'][len(pricedata)-1]))
	print("ColFinal: " + colFinal)
	print("ColFinal2: " + colFinal2)
	
		
	print(str(dt.datetime.now()) + "	 " + timeframe + " Update Function Completed.\n")

# This function places a market order in the direction BuySell, "B" = Buy, "S" = Sell, uses symbol, amount, stop, limit
def enter(BuySell):
	direction = True;
	if BuySell == "S":
		direction = False;
	try:
		opentrade = con.open_trade(symbol=symbol, is_buy=direction,amount=amount, time_in_force='GTC',order_type='AtMarket',is_in_pips=True,limit=limit, stop=stop, trailing_step=trailstop_value)
	except:
		print("	  Error Opening Trade.")
	else:
		print("	  Trade Opened Successfully.")

# This function closes all positions that are in the direction BuySell, "B" = Close All Buy Positions, "S" = Close All Sell Positions, uses symbol
def exit(BuySell=None):
	openpositions = con.get_open_positions(kind='list')
	isbuy = True
	if BuySell == "S":
		isbuy = False
	for position in openpositions:
		if position['currency'] == symbol:
			if BuySell is None or position['isBuy'] == isbuy:
				print("	  Closing tradeID: " + position['tradeId'])
				try:
					closetrade = con.close_trade(trade_id=position['tradeId'], amount=position['amountK'])
				except:
					print("	  Error Closing Trade.")
				else:
					print("	  Trade Closed Successfully.")

# Returns number of Open Positions for symbol in the direction BuySell, returns total number of both Buy and Sell positions if no direction is specified
def countOpenTrades(BuySell=None):		
	openpositions = con.get_open_positions(kind='list')
	isbuy = True
	counter = 0
	if BuySell == "S":
		isbuy = False
	for position in openpositions:
		if position['currency'] == symbol:
			if BuySell is None or position['isBuy'] == isbuy:
				counter+=1
	return counter

Prepare() # Initialize strategy
StrategyHeartBeat() # Run strategy

SyntaxError: invalid syntax (<ipython-input-1-e359cb888d1f>, line 125)